<a href="https://colab.research.google.com/github/sanAkel/ocean-hurricane/blob/main/prep_data/RTOFS/download_surf_RTOFSv2p4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download RTOFS version 2.4 (`v2p4`) data from [this URL.](https://noaa-nws-rtofs-pds.s3.amazonaws.com/index.html)

## For HAFS, [this is the first available MOM6 output.](https://noaa-nws-hafs-pds.s3.amazonaws.com/index.html#hfsa/20240822/00/)

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

### User inputs

In [ ]:
system, version = ["rtofs", "v2.4"]

start_date, end_date = ["20240703", "20240717"]

hSkip=1 # skip hour
hours = np.arange(0, 25, hSkip)

In [ ]:
if system == "rtofs":
  bucket = "noaa-nws-rtofs-pds"
  oPath = f"drive/MyDrive/datasets/RTOFS/{version}/"
elif system == "hafs":
  bucket = "noaa-nws-hafs-pds"
else:
  print('UNKNOWN/not yet coded bucket! Fix and try again.')

url_base = f"https://{bucket}.s3.amazonaws.com/{system}."

In [ ]:
data_dates = pd.date_range(start=start_date, end=end_date)

url = ""
for dd in data_dates:
  dSTR = dd.strftime("%Y%m%d")
  print(f"Downloading data for:\t {dSTR}")
  url = url_base + dd.strftime("%Y%m%d") + "/" + system+"_glo.t00z."

  !mkdir -p ./{dSTR}
  for hr in hours:
    if hr == 0:
      a_file, b_file = ["n00.archs.a.tgz", "n00.archs.b"]
    else:
      a_file, b_file = ["n-"+str(hr).zfill(2) + ".archs.a.tgz",\
                        "n-"+str(hr).zfill(2) + ".archs.b"]
    a_file_url = url + a_file
    b_file_url = url + b_file

    a_file_tar = dd.strftime("%Y%m%d") + "_" + a_file
    b_file = system+"_glo.t00z." + b_file

    #print(a_file_url, b_file_url)
    !wget -q $a_file_url -O $a_file_tar
    !wget -q $b_file_url -O $b_file

    !tar -xzf $a_file_tar -C ./{dSTR}
    !mv $b_file ./{dSTR}

  !mv ./{dSTR} $oPath
  !rm -f *.tgz